# Guardar conjuntos de datos de DEPMAP20Q3 en tablas BigQuery 
```
Título original: How to save DEPMAP20Q3 datasets in BigQuery Tables
Autor:  Bahar Tercan
Creado: 02-08-2022
Cuaderno de Jupyter traducido y modificado por Martín Sende Pombo.
Fecha de modificación: 24-02-2024
Propósito: Descargar datos del proyecto DEPMAP20Q3 en tablas BigQuery. 
Notas: MyBinder puede reiniciar el kernel debido al largo tiempo de ejecución de este cuaderno, por lo que solo recomendamos la ejecución local para este cuaderno.
```

Este cuaderno proporciona código para guardar los datos de DepMap_public_20Q3 en tablas de bigquery.
Los usuarios no necesitan ejecutar esta "pipeline" para obtener los datos, sino que esta tubería muestra como guardamos los datos en las tablas de bigquery.

Por favor, póngase en contacto con Bahar Tercan, btercan@systemsbiology.org, si tiene más preguntas sobre este cuaderno.

## Instalación e importación de las bibliotecas requeridas

In [ ]:
# Por favor, no ejecute este bloque de código si está corriendo el cuaderno en MyBinder.
# Este bloque de código instala las dependencias. Por favor, descoméntelo y ejecútelo sólo una vez, la primera ocasión que ejecute este cuaderno en su ordenador.
# (Si ya ha ejecutado este bloque para el "pipeline" shRNA_save_data, no necesita correrlo).
#!pip3 install numpy
#!pip3 install pandas
#!pip3 install google.cloud
#!pip3 install pandas_gbq
#!pip3 install importlib

In [ ]:
import numpy as np
import sys
sys.path.append('../../Scripts/')
import importlib
import BIGQUERY_operations
importlib.reload(BIGQUERY_operations)
from BIGQUERY_operations import *
import DEPMAP_data_preprocessing
importlib.reload(DEPMAP_data_preprocessing)
from DEPMAP_data_preprocessing import *


## Autenticación de Google
El primer paso es autorizar el acceso a BigQuery y a Google Cloud. Para más información vea ['Quick Start Guide to ISB-CGC'](https://isb-cancer-genomics-cloud.readthedocs.io/en/latest/sections/HowToGetStartedonISB-CGC.html) y puede encontrar métodos de autentificación alternativos [aquí](https://googleapis.dev/python/google-api-core/latest/auth.html).

Además necesita [crear un proyecto de Google Cloud](https://cloud.google.com/resource-manager/docs/creating-managing-projects#console) para poder ejecutar consultas BigQuery.

### * Si está ejecutando este cuaderno en un entorno de máquina virtual de Google (Compute Engine, App Engine, Cloud Run, Cloud Functions): 
La autenticación debería "simplemente funcionar".

### * Si la ejecución se realiza en un entorno local: 
La forma más sencilla de autenticarse es emplear el SDK de Google Cloud.

In [ ]:
#Por favor, no ejecute este bloque de código si está corriendo el cuaderno en MyBinder.
!gcloud auth application-default login

###

In [ ]:
from google.cloud import bigquery

In [ ]:
# Configurar la información del proyecto y el cliente bigquery.
# Por favor, reemplace syntheticlethality por el ID de su propio proyecto.


project_id='syntheticlethality'

# Construir un objeto cliente BigQuery.
client = bigquery.Client(project_id)

## Definición del nombre y la descripción del conjunto de datos

In [ ]:
dataset_name='DepMap_public_20Q3_backup'
dataset_description="""  
Esta versión de DepMap contiene datos de cribados CRISPR knockout del proyecto Achilles, así como datos de caracterización genómica del proyecto CCLE.

Referencias:
Dempster, J.M., Rossen, J., Kazachkova, M., Pan, J., Kugener, G., Root, D.E., and Tsherniak, A. (2019). Extracting Biological Insights from the Project Achilles Genome-Scale CRISPR Screens in Cancer Cell Lines.

Meyers, R.M., Bryan, J.G., McFarland, J.M., Weir, B.A., Sizemore, A.E., Xu, H., Dharia, N.V., Montgomery, P.G., Cowley, G.S., Pantel, S., et al. (2017). Computational correction of copy number effect improves specificity of CRISPR-Cas9 essentiality screens in cancer cells. Nat. Genet. 49, 1779–1784.

Ghandi, M., Huang, F.W., Jané-Valbuena, J., Kryukov, G.V., Lo, C.C., McDonald, E.R., 3rd, Barretina, J., Gelfand, E.T., Bielski, C.M., Li, H., et al. (2019). Next-generation characterization of the Cancer Cell Line Encyclopedia. Nature 569, 503–508
"""

CreateDataSet(client, dataset_name, project_id, dataset_description)

## Descarga de los conjuntos de datos DEPMAP20Q3 del portal DEPMAP

In [ ]:
#CCLE_mutations.csv
mutation_data=pd.read_csv("https://ndownloader.figshare.com/files/24613355", sep="\t") 
#https://ndownloader.figshare.com/files/24613355

#sample_info.csv
sample_info=pd.read_csv("https://ndownloader.figshare.com/files/24613394",  sep=",")
#https://ndownloader.figshare.com/files/24613394

#CCLE_gene_cn.csv
cn_data=pd.read_csv("https://ndownloader.figshare.com/files/24613352", index_col=0) #Índice: | ACH-000001
#https://ndownloader.figshare.com/files/24613352

#CCLE_expression.csv
gene_exp_data=pd.read_csv("https://ndownloader.figshare.com/files/24613325", index_col=0) #Índice: | ACH-001636
#https://ndownloader.figshare.com/files/24613325

#Achilles_gene_effect.csv
achilles_gene_effect=pd.read_csv("https://ndownloader.figshare.com/files/24613292", index_col=0) #Índice: DepMap_ID | ACH-000004
#https://ndownloader.figshare.com/files/24613292


## Lectura de las anotaciones creadas manualmente para las columnas de las tablas

In [ ]:
# Obtener anotaciones del archivo excel
depmap_annotations=pd.ExcelFile("../Depmap20Q3_annotation.xlsx")

In [ ]:
mutation_annotation=depmap_annotations.parse('CCLE_Mutations')
sample_annotation=depmap_annotations.parse('Sample_Info')
achilles_gene_effect_annotation=depmap_annotations.parse('Achilles_Gene_Effect')
CCLE_expression_annotation=depmap_annotations.parse('CCLE_Gene_Expression')
cnv_annotations=depmap_annotations.parse('CCLE_Copy_Number')

## Creación del conjunto de datos de BigQuery en el proyecto de Google Cloud

In [ ]:
dataset_name='DepMap_public_20Q3_backup'
dataset_description="""  
Esta versión de DepMap contiene datos de cribados CRISPR knockout del proyecto Achilles, así como datos de caracterización genómica del proyecto CCLE.

Referencias:
Dempster, J.M., Rossen, J., Kazachkova, M., Pan, J., Kugener, G., Root, D.E., and Tsherniak, A. (2019). Extracting Biological Insights from the Project Achilles Genome-Scale CRISPR Screens in Cancer Cell Lines.

Meyers, R.M., Bryan, J.G., McFarland, J.M., Weir, B.A., Sizemore, A.E., Xu, H., Dharia, N.V., Montgomery, P.G., Cowley, G.S., Pantel, S., et al. (2017). Computational correction of copy number effect improves specificity of CRISPR-Cas9 essentiality screens in cancer cells. Nat. Genet. 49, 1779–1784.

Ghandi, M., Huang, F.W., Jané-Valbuena, J., Kryukov, G.V., Lo, C.C., McDonald, E.R., 3rd, Barretina, J., Gelfand, E.T., Bielski, C.M., Li, H., et al. (2019). Next-generation characterization of the Cancer Cell Line Encyclopedia. Nature 569, 503–508
"""

CreateDataSet(client, dataset_name, project_id, dataset_description)

## Guardar los datos de mutación en una tabla de BigQuery

In [ ]:
mutation_table_name='CCLE_mutation'
mutation_table_desc='''Pipeline: MAF de las mutaciones genéticas. Archivo original: CCLE_mutations.csv
Enlace de descarga: https://ndownloader.figshare.com/files/24613355.'''

mutation_dict=mutation_annotation.to_dict('records')
mutation_table=CreateTable(client, mutation_data, dataset_name, mutation_table_name, project_id,  mutation_table_desc, mutation_dict)

## Guardar datos de información de las muestras en una tabla de BigQuery

In [ ]:
sample_info_table_name="sample_info"
sample_info_table_desc='''Definiciones de información sobre líneas celulares. Archivo original: sample_info.csv 
Enlace de descarga: https://ndownloader.figshare.com/files/24613394'''

sample_dict=sample_annotation.to_dict('records')
CreateTable(client, sample_info, dataset_name, sample_info_table_name, project_id, sample_info_table_desc, sample_dict)

## Guardar los datos del número de copia en una tabla de BigQuery

In [ ]:
cn_table_desc='''Tubería: datos del número de copias a nivel de genes, transformados aplicando el log2 con un pseudoconteo de 1. Esto se genera mapeando los genes en las llamadas a nivel de segmento. 
Archivo original: WES_SNP_CN_data.csv Enlace de descarga: https://ndownloader.figshare.com/files/24613352.'''

cnv_long_format=CRISPRPreprocess(cn_data, 'CNA')
cnv_long_format['Entrez_ID']=pd.to_numeric(cnv_long_format['Entrez_ID'])
cnv_table_name="CCLE_gene_cn"
cnv_dict=cnv_annotations.to_dict('records')
CreateTable(client, cnv_long_format, dataset_name, cnv_table_name, project_id, cn_table_desc, cnv_dict)

## Guardar los datos de expresión génica en una tabla de BigQuery

In [ ]:
CCLE_expression_table_desc='''
Pipeline: Datos de expresión génica de RNAseq TPM sólo para genes codificantes de proteínas utilizando RSEM. 
Transformación aplicando el Log2, utilizando un pseudoconteo de 1. Archivo original: CCLE_expression.csv 
Enlace de descarga: https://ndownloader.figshare.com/files/24613325'''

CCLE_expression_long_format=CRISPRPreprocess(gene_exp_data, 'TPM')
CCLE_expression_long_format['Entrez_ID']=pd.to_numeric(CCLE_expression_long_format['Entrez_ID'])
CCLE_expression_table_name="CCLE_gene_expression"
CCLE_expression_dict=CCLE_expression_annotation.to_dict('records')
CreateTable(client, CCLE_expression_long_format, dataset_name, CCLE_expression_table_name, project_id, CCLE_expression_table_desc, CCLE_expression_dict)




## Guardar datos de efectos genéticos en una tabla de BigQuery

In [ ]:
achilles_gene_effect_table_desc='''Tubería: Achilles_Post-CERES_ Datos CERES con componentes principales fuertemente relacionados con efectos de lote conocidos eliminados, luego desplazados y escalados por línea celular para que la mediana del efecto KO no esencial sea 0 y la mediana del efecto KO esencial sea -1.
Archivo original: Achilles_gene_effect.csv 
Enlace de descarga: https://ndownloader.figshare.com/files/24613352
'''
achilles_gene_effect_long_format=CRISPRPreprocess(achilles_gene_effect, 'Gene_Effect')
achilles_gene_effect_long_format['Entrez_ID']=pd.to_numeric(achilles_gene_effect_long_format['Entrez_ID'])
achilles_gene_effect_table_name="Achilles_gene_effect"
achilles_gene_effect_dict=achilles_gene_effect_annotation.to_dict('records')
CreateTable(client, achilles_gene_effect_long_format, dataset_name, achilles_gene_effect_table_name, project_id, achilles_gene_effect_table_desc, achilles_gene_effect_dict)

